In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Adam
import torch.nn.functional as F

import csv
from skimage import io

from PIL import Image
import pandas as pd

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
import time
import os
import copy

import sys
sys.path.append('JupyterProjects/EYE-SEA/DataSets/MARVEL_code')

import import_ipynb
#import Dataset_loader_MARVEL
import ResNetCaps
import Pets_Loader
import Animals_Loader

verbose = False
USE_CUDA = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

CIFAR10_USE = False
MARVEL_USE = False
PETS_USE = False
Animals_USE = True

In [ ]:
class MARVEL_dataset(Dataset):
    def __init__(self, dat_file,train = True, transform = None):   
        self.root_dir = os.path.dirname(dat_file)
        datContent = [i.strip().split(',') for i in open(dat_file).readlines()]
        if train:
            csv_file = os.path.join(self.root_dir, "data_Train.csv")
        else:
            csv_file = os.path.join(self.root_dir, "data_Test.csv")
        with open(csv_file, "w") as f:
            writer = csv.writer(f,delimiter=',')
            writer.writerow(["counter", "set", "class", "label","location"])
            for line in datContent:
                if train and line[1]=='1':
                    if not(line[4] == '-'):
                        writer.writerows([line])  
                if not(train) and line[1] == '2':
                    if not(line[4]=='-'):
                        writer.writerows([line]) 
                
        self.MARVEL_datafile = pd.read_csv(csv_file)       
        self.transform = transform
        
    def __len__(self):
        return len(self.MARVEL_datafile)
    
    def __getitem__(self,idx):
        img_name = self.MARVEL_datafile.iloc[idx,4]
        image = self.__loadfile(img_name)
        target = self.MARVEL_datafile.iloc[idx,2]
        if self.transform:
            image = Image.fromarray(image)
            sample = self.transform(image)
        else:
            sample = image
        return (sample,target)
    
    def __loadfile(self, data_file):
        image = io.imread(data_file)
        if len(image.shape)<3:
            image = np.stack((image,)*3, axis=-1)
        return image

In [ ]:
dataset_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),        
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])


batch_size = 32


if CIFAR10_USE: 
    NUM_CLASSES = 10
    print("CIFAR10")
    image_datasets = {'train': datasets.CIFAR10('../data', train=True, download=True, transform=dataset_transform),'val': datasets.CIFAR10('../data', train=False, download=True, transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")

if MARVEL_USE: 
    NUM_CLASSES = 26
    print("MARVEL")
    dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/marveldataset2016-master/FINAL.dat"

    image_datasets = {'train': MARVEL_dataset(dat_file,train = True,transform=dataset_transform),'val': MARVEL_dataset(dat_file,train = False,transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")
    
if  PETS_USE:
    NUM_CLASSES = 37
    print("PETS")
    dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/Pets/Pet_Datasets"

    image_datasets = {'train': Pets_Loader.PETS_dataset(dat_file,train = True,transform=dataset_transform),'val': Pets_Loader.PETS_dataset(dat_file,train = False,transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")    
    
if  Animals_USE:
    NUM_CLASSES = 50
    print("Animals")
    dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/Animals_with_Attributes2/JPEGImages"

    image_datasets = {'train': Animals_Loader.Animals_dataset(dat_file,train = True,transform=dataset_transform),'val': Animals_Loader.Animals_dataset(dat_file,train = False,transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")    

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [ ]:
def shift_left_once(A):
    a =  []
    AD = A.detach().cpu().numpy()
    temp = AD[0,:]

    for index in range(1,len(AD)):
        a = np.concatenate([a,AD[index]])
    a = np.concatenate([a,temp])
    a_tensor = torch.from_numpy(a)
    return ((a_tensor.view(len(a),1)).float()).to(device)

def shift_left(lst, n):
    assert (n >= 0), "n should be non-negative integer"
    for _ in range(n):
        shift_left_once(lst)

def circulant(A):
    r = A.size()[0]
    circ_list_a = []
    circ_list_a.append(A)
    
    a = shift_left_once(A)
    for i in range(r-1):   
        circ_list_a.append(a)      
        a = shift_left_once(a)

    
    circ = torch.cat(circ_list_a,0).detach().requires_grad_()
    circ = circ.view([r,r])
    if verbose: print("circular matrix {}".format(circ))
    return circ.to(device)
        
        
    

In [ ]:
class Interaction(nn.Module):
    def __init__(self,NUM_CLASSES):
        super(Interaction, self).__init__()
        self.modelCaps1 = ResNetCaps.ResNetCaps(NUM_CLASSES)
        self.modelCaps2 = ResNetCaps.ResNetCaps(NUM_CLASSES)
        
        self.W1 = torch.randn(NUM_CLASSES,NUM_CLASSES)
        self.W1.requires_grad = True
        
        self.W2 = torch.randn(NUM_CLASSES,NUM_CLASSES)
        self.W2.requires_grad = True
        
        self.modelLin = nn.Linear(NUM_CLASSES, NUM_CLASSES)
        self.NClass = NUM_CLASSES

    def cuda(self, device=None):
        self = super().cuda(device)
        self.W1 = self.W1.cuda(device)
        self.W2 = self.W2.cuda(device)
        return self 
        
    def forward(self,inputs):
        digit1,_ = self.modelCaps1(inputs)
        digit2,_ = self.modelCaps2(inputs)
        
        output = self.modelLin(self.interaction(digit1,digit2))
        return output

    def model_loss(self, x, labels):
        batch_size = x.size(0)

        v_c = x
        if verbose: print("v_c {}".format(v_c.size()))
        left = F.relu(0.9 - v_c).view(batch_size, -1)
        right = F.relu(v_c - 0.1).view(batch_size, -1)

        loss = labels * left + 0.5 * (1.0 - labels) * right

        loss = loss.sum(dim=1).mean()
        
        return loss

    def interaction(self,x,y):
        
        self.x = x
        self.y = y
         
        Z_list = []
        labels_size = x.size()[1]
        batch = min(x.size()[0],batch_size)
        for i in range(batch):
        
            xi = torch.sum(x[i,:,:,0],dim=1)
            yi = torch.sum(y[i,:,:,0],dim=1)
            
            xi = xi.view(self.NClass,1)
            yi = yi.view(self.NClass,1)

            V = torch.mm(self.W1,xi)   #< questo fa batch-matrix con matrix ?
            C = torch.mm(self.W2,yi)

            #CIRCOLANTE
            A = circulant(V)
            B = circulant(C)

            #INTERACTION MOMENT
            F = torch.mm(B,V)
            G = torch.mm(A,C)
            
            M = torch.add(F,G)

            Z_list.append(M)
            
        Z = torch.cat(Z_list,0)
        if verbose : print(Z.size())
        Z = Z.view(batch,labels_size)
        
        return Z   
        
    

In [ ]:
def lr_decrease(optimizer, lr_clip):  
    for param_group in optimizer.param_groups:
        init_lr = param_group['lr'] 
        param_group['lr'] = init_lr*lr_clip
        
def isnan(x):
    return x != x        


In [ ]:
lr = 0.00001
threshold = 30
lr_clip = 0.01

In [ ]:
model = Interaction(NUM_CLASSES)
#model = model.to(device)
model = model.cuda(device)#<for weights matrix

#optimizers
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr = lr)
n_epochs = 100
#train
start = time.time()
#batch_id = 100
#inputs, labels = next(iter(dataloaders['train']))
accuracy_train = []
loss_train = []


for epoch in range(n_epochs): 
    model.train() 
    train_loss = 0
    train_accuracy = 0

    print('epoch {}:{}'.format(epoch+1, n_epochs)) 
    for batch_id, (inputs, labels) in enumerate(dataloaders['train']):
        if MARVEL_USE: labels = labels-1
        labels =torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)
        inputs, labels = Variable(inputs), Variable(labels)
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        _,label = torch.max(labels, 1)

        loss = criterion(outputs, label.long())
        if isnan(loss): 
            request_exit = True
            print("lost loss")
            break
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        train_accuracy += (sum(np.argmax(outputs.data.cpu().numpy(), 1) == np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))

        if batch_id % 100 == 0:
            print("train accuracy:", sum(np.argmax(outputs.data.cpu().numpy(), 1) == 
                                   np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
            print("loss : {}".format(loss.data[0]))
            if verbose: print("masked {}".format(np.argmax(masked.data.cpu().numpy(), 1)))
            if verbose: print("labels {}".format(np.argmax(labels.data.cpu().numpy(), 1)))
        #                batch_accuracy.append(sum(np.argmax(preds.data.cpu().numpy(), 1) == 
        #                                       np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
    accuracy_train.append(train_accuracy/batch_id)#len(dataloaders['train']))
    loss_train.append(train_loss/batch_id)#len(dataloaders['train']))
    
    if epoch>2:
        if accuracy_train[-1]<(accuracy_train[-2]*0.5):
            print("Learnining rate decrease")
            lr_decrease(optimizer,lr_clip)
   # if epoch == threshold:
    #    print("Learnining rate decrease")
     #   lr_decrease(optimizer, lr_clip)

end = time.time()
print("Training time execution {}".format(end-start))
print("Loss value for training phase: {}".format(train_loss / batch_id))#len(dataloaders['train'])))
print("Accuracy value for training phase: {}".format(train_accuracy / batch_id))#len(dataloaders['train'])))

In [ ]:
print("Training time execution {}".format(end-start))
print("Loss value for training phase: {}".format(train_loss / batch_id))#len(dataloaders['train'])))
print("Accuracy value for training phase: {}".format(train_accuracy / batch_id))#len(dataloaders['train'])))

In [ ]:
accuracy_train[-1]=train_accuracy/batch_id#len(dataloaders['train']))
loss_train[-1]=(train_loss/batch_id)#len(dataloaders['train']))


In [ ]:
epochs = np.arange(1,n_epochs+1)
plt.plot(epochs, loss_train, color='g')
plt.plot(epochs, accuracy_train, color='orange')
plt.xlabel('Epochs')
plt.ylabel('Accuracy - Loss')
plt.title('Training phase')
plt.show() 

In [ ]:
model.eval()
test_loss = 0
test_accuracy = 0 
start = time.time()
for batch_id, (inputs, labels) in enumerate(dataloaders['val']):
    if MARVEL_USE: labels = labels-1

    labels = torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)
    inputs, labels = Variable(inputs), Variable(labels)

    if USE_CUDA: inputs, labels = inputs.to(device), labels.to(device)#cuda()

    outputs = model(inputs)
    _,label = torch.max(labels, 1)
    loss = criterion(outputs, label.long())

    test_loss += loss.data[0]
    test_accuracy += (sum(np.argmax(outputs.data.cpu().numpy(), 1) == np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))

    if batch_id % 100 == 0:
        print("test accuracy:", sum(np.argmax(outputs.data.cpu().numpy(), 1) == 
                               np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
end = time.time()
print("Test time execution {}".format(end-start))
print("Loss value for test phase: {}".format(test_loss /  len(dataloaders['val']))) 
print("Accuracy value for test phase: {}".format(test_accuracy / len(dataloaders['val'])))

In [ ]:
torch.cuda.empty_cache()